## Carga Data AUDIENCIAS

In [1]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm

from src.data import cleandata

In [2]:
path_raw = "../data/raw/pjud" 
archivos = os.listdir(path_raw) 
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
# Creacion dataframe con los datos de Audiencias Realizadas Penal de los años 2015 a 2019

dataframes = []

for archivo in archivos:
    if archivo.find("Audiencias Realizadas Penal") != -1:
        df = pd.read_csv(f"{path_raw}/{archivo}", sep = ";", encoding = 'cp850', dtype = 'unicode', low_memory = True)
        dataframes.append(df) 
        
df_audiencias = pd.concat(dataframes, axis = 0)


In [4]:
df_audiencias.rename(columns = {'CÓD. CORTE':'COD. CORTE',
                                'CÓD. TRIBUNAL':'COD. TRIBUNAL',
                                'DURACIÓN  AUDIENCIA':'DURACION AUDIENCIA',
                                'AGENDAMIENTO (DÍAS CORRIDOS)':'DIAS AGENDAMIENTO',
                                'DURACIÓN AUDIENCIA (MINUTOS)':'DURACION AUDIENCIA (MIN)',
                                'FECHA PROGRAMACIÓN AUDIENCIA':'FECHA PROGRAMACION AUDIENCIA'
                               },
                     inplace = True)

# TRANSFORMAMOS DE FLOAT A INTEGER

df_audiencias['COD. CORTE'] = df_audiencias['COD. CORTE'].fillna(0).astype(np.int16)
df_audiencias['COD. TRIBUNAL'] = df_audiencias['COD. TRIBUNAL'].fillna(0).astype(np.int16)

In [5]:
# Podemos observar que existen columnas que se repiten, y que tienen datos NAN en algunas pero esos datos 
# en otras columnas, pasa en TIPO AUDIENCIA=TIPO DE AUDIENCIA, AGENDAMIENTO (DÍAS CORRIDOS)=PLAZO AGENDAMIENTO
# (DÍAS CORRIDOS), DURACIÓN AUDIENCIA (MINUTOS)= DURACIÓN AUDIENCIA

df_audiencias['TIPO DE AUDIENCIA'] = df_audiencias['TIPO DE AUDIENCIA'].fillna(df_audiencias['TIPO AUDIENCIA'])
df_audiencias['DIAS AGENDAMIENTO'] = df_audiencias['DIAS AGENDAMIENTO'].fillna(df_audiencias['PLAZO AGENDAMIENTO (DIAS CORRIDOS)']).astype(np.int16)
df_audiencias['DURACION AUDIENCIA (MIN)'] = df_audiencias['DURACION AUDIENCIA (MIN)'].fillna(df_audiencias['DURACION AUDIENCIA'])

In [6]:
# Elimino las columnas reemplazadas

df_audiencias.drop(['TIPO AUDIENCIA','PLAZO AGENDAMIENTO (DIAS CORRIDOS)','DURACION AUDIENCIA','TOTAL AUDIENCIAS'], 
                   axis = 'columns', inplace = True)

In [7]:
# Transformamos fechas

df_audiencias['FECHA PROGRAMACION AUDIENCIA'] = df_audiencias['FECHA PROGRAMACION AUDIENCIA'].progress_apply(cleandata.convierte_fecha)
df_audiencias['FECHA AUDIENCIA'] = df_audiencias['FECHA AUDIENCIA'].progress_apply(cleandata.convierte_fecha)

100%|██████████| 6712514/6712514 [00:11<00:00, 571783.74it/s]


In [8]:
# Elimino espacios en las columnas tipo objetos

df_audiencias = df_audiencias.progress_apply(cleandata.elimina_espacios, axis=0)

100%|██████████| 11/11 [00:53<00:00,  4.84s/it]


In [9]:
# Elimino tildes 

cols = df_audiencias.select_dtypes(include = ["object"]).columns
df_audiencias[cols] = df_audiencias[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 6/6 [01:36<00:00, 16.11s/it]


In [10]:
# Categorizar

df_audiencias['CORTE'] = df_audiencias['CORTE'].astype('category')
df_audiencias['TIPO CAUSA'] = df_audiencias['TIPO CAUSA'].astype('category')

In [11]:
# Reset el index para realizar feather

df_audiencias.reset_index(inplace = True)
df_audiencias

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,TIPO DE AUDIENCIA,FECHA PROGRAMACION AUDIENCIA,FECHA AUDIENCIA,DIAS AGENDAMIENTO,DURACION AUDIENCIA (MIN)
0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,AUDIENCIA DE JUICIO ORAL TOP,2018-04-03,2018-05-28,55,164
1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,AUDIENCIA DE LECTURA DE SENTENCIA.,2018-06-01,2018-06-01,0,9
2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,AUDIENCIA DE MEDIDAS CAUTELARES.,2018-06-19,2018-06-23,4,4
3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,AUDIENCIA DE COMPARECENCIA JUDICIAL.,2018-04-03,2018-05-23,50,3
4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,AUDIENCIA PRESENTACION VOL.IMPUTADO C/ORD. DETEN.,2018-05-24,2018-05-24,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
6712509,1305380,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,85-2015,Ordinaria,AUDIENCIA DE LECTURA DE SENTENCIA.,2015-12-21,2015-12-24,3,3
6712510,1305381,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,90-2013,Ordinaria,AUDIENCIA DE JUICIO ORAL TOP,2015-07-06,2015-08-12,37,197
6712511,1305382,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,90-2013,Ordinaria,AUDIENCIA DE LECTURA DE SENTENCIA.,2015-08-12,2015-08-17,5,3
6712512,1305383,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,92-2015,Ordinaria,AUDIENCIA DE COMPARECENCIA JUDICIAL.,2015-12-16,2015-12-21,5,7


In [12]:
# Guardamos dataset como archivo feather
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

df_audiencias.to_feather(f'{path_interim}/Audiencias_feather')